# Dask dataframe example

This is an introduction to using Dask dataframes to run out-of-core computations on datasets.


In [ ]:
# Imports
# Get number of CPUS allocated to Server
import os
num_cpus = os.environ.get('NUM_CPUS')
if num_cpus is None:
    num_cpus = os.cpu_count()
else:
    num_cpus = int(num_cpus)

print('Number of CPUs in server: {}'.format(num_cpus))

import dask
from dask import dataframe as dd
from dask.diagnostics import CacheProfiler, ResourceProfiler, Profiler, ProgressBar, visualize

# Set up default number of workers and enable global progress bar
dask.set_options(num_workers=num_cpus)
progress_bar = ProgressBar()
progress_bar.register()

# Set up bokeh JS for visualization
from bokeh.io import output_notebook
output_notebook()

%matplotlib inline
import matplotlib.pyplot as plt

# Create an anonymous S3 client to download data
import boto3
from botocore import UNSIGNED
from botocore.client import Config

s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

## Flight data from USA domestic flights 2001 - 2008

We will explore a dataset that contains information on every domestic flight in the USA between 2001 and 2008. Every year is a CSV file of ~700MB, so the compilation of all the years is at the level of annoying data: data that will typically not fit on a single node but is not large enough to warrant the overhead of Big Data technologies.

We read all CSVs with a wildcard into a Dask Dataframe and select a few columns. Note that this is a delayed operation, the whole file has not been actually read when running `read_csv`, but only the column names and types have been inferred from the first few lines of the file.

In [ ]:
years_to_analyse = range(2001, 2008) # each year is a ~700 MB CSV

In [ ]:
# Download data
print('This might take a while, downloading up to {} MB of data.'
      .format(700 * len(years_to_analyse)))
for year in years_to_analyse:
    filename = '{}.csv'.format(year)
    if not os.path.exists(filename):
        print('Downloading {}.csv...'.format(year))
        s3.download_file('dask-data',
                         'airline-data/{}.csv'.format(year),
                         filename)
    else:
        print('File {} already exists, not downloading.'.format(filename))
            

In [ ]:
cols = ['Year', 'Month', 'DayOfWeek', 'DepDelay',
        'CRSDepTime', 'UniqueCarrier', 'Origin', 'Dest']
# Create the dataframe reading CSVs from disk,
# and internally split them into 128MB chunks
df = dd.read_csv('*.csv', usecols=cols, blocksize=int(128e6))

The `df` object contains the graph that dask will execute when the operation is computed. You can visualize it by calling the `visualize()` method. You may need to install both the graphviz python package and system package for this to work (run `source activate Python3 && pip install graphviz && sudo apt-get install -y graphviz` in a terminal and restart this notebook's kernel).

In [ ]:
df.visualize(rankdir='LR')

Now we can start to define operations that will be run on the dataframe. As above, these commands do not execute the analysis, only define an execution graph.

We build two new columns: 
    - `hour`: the hour at which the flight departed
    - `delayed`: whether the flight had more than 15 minutes delay.
    
On this new dataframe, we perform three aggregations: delay fraction by month, delay fraction by hour, and delay fraction by carrier.

In [ ]:
df2 = (df.drop(['DepDelay', 'CRSDepTime'], axis=1)
        .assign(hour=df.CRSDepTime.clip(upper=2399)//100,
                delayed=(df.DepDelay.fillna(16) > 15).astype('float')))
# Define some aggregations to plot
aggregations = [
    df2.groupby('Year').delayed.mean(),
    df2.groupby('Month').delayed.mean(),
    df2.groupby('hour').delayed.mean(),
    df2.groupby('UniqueCarrier').delayed.mean().nlargest(20),
]

Now we can visualize the execution graph of all the above operations that will be needed to carry out the computation of delayed fraction by carrier.

In [ ]:
delayed_by_carrier_dask = aggregations[3]
delayed_by_carrier_dask.visualize(rankdir='LR')

Finally, we can perform the actual computation! On any dask object you can call `.compute()` to obtain the computed value and execute the graph. Because here we have three aggregations that will use the same data, we group them in a single call to `dask.compute`, which helps optimize the reuse of intermediate data. 

In [ ]:
with Profiler() as prof, ResourceProfiler(dt=0.25) as rprof, CacheProfiler() as cprof, ProgressBar():
    # Compute them all in a single pass over the data
    (delayed_by_hour,
     delayed_by_month,
     delayed_by_year,
     delayed_by_carrier) = dask.compute(*aggregations, num_workers=num_cpus)

Having wrapped the execution in the `with Profiler() ... ResourceProfiler()` context managers now allows us to visualize the resource usage of the execution. 

In [ ]:
visualize([prof, rprof, cprof])

Finally, we can take a look at the aggregated metrics, which are regular pandas dataframes:

In [ ]:
for df in [delayed_by_hour, delayed_by_month, delayed_by_year]:
    f, ax = plt.subplots()
    df.sort_index(inplace=True)
    ax.plot(df.index, df.values, '-o')
    ax.set_title('Delay fraction grouped by {}'.format(df.index.name))
    ax.set_xlabel(df.index.name)
    ax.set_ylabel('Fraction ' + df.name)

In [ ]:
delayed_by_carrier.sort_values(ascending=False).plot(kind='bar')

# Other medium sized datasets


These are some medium sized data that are good examples of when dask is needed.

NYCTaxi
------

[Download link](http://www.andresmh.com/nyctaxitrips/)

Taxi trips taken in 2013 released by a FOIA request.  Around 20GB CSV uncompressed.

**Try the following:**

*  Use `dask.dataframe` with pandas-style queries
*  Store in HDF5 both with and without categoricals, measure the size of the file and query times
*  Set the index by one of the date-time columns and store in castra (also using categoricals).  Perform range queries and measure speed.  What size and complexity of query can you perform while still having an "interactive" experience?

Github Archive
----------------

[Download link](https://www.githubarchive.org/)

Every public github event for the last few years stored as gzip compressed line-delimited JSON data.  Watch out, the schema switches at the 2014-2015 transition.

**Try the following:**

*  Use `dask.bag` to inspect the data
*  Drill down using functions like `pluck` and `filter`
*  Find who the most popular committers were in 2015

Reddit Comments
-----------------

[Download link](https://www.reddit.com/r/datasets/comments/3bxlg7/i_have_every_publicly_available_reddit_comment/)

Every publicly available reddit comment, available as a large torrent

**Try the following:**

*  Use `dask.bag` to inspect the data
*  Combine `dask.bag` with `nltk` or `gensim` to perform textual analyis on the data
*  Reproduce the work of [Daniel Rodriguez](http://danielfrg.com/blog/2015/07/21/reproduceit-reddit-word-count-dask/) and see if you can improve upon his speeds when analyzing this data.

NYC 311
---------

[Download link](https://nycopendata.socrata.com/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9)

All 311 service requests since 2010 in New York City

European Centre for Medium Range Weather Forecasts
----------------------------------------------------------

[Download script](https://gist.github.com/mrocklin/26d8323f9a8a6a75fce0)

Download historical global weather data from the ECMWF.

**Try the following:**

*  What is the variance in temperature over time?
*  What areas experienced the largest temperature swings in the last month relative to their previous history?
*  Plot the temperature of the earth as a function of latitude and then as longitude